# Análise Comparativa de Modelos

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## 1. Obtenção de Dados

#### Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pre-processamento.

In [2]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,survived,se o individuo esta vivo ou morto,qualitativa,nominal
1,pclass,Classe de ingresso,qualitativa,ordinal
2,sex,sexo,qualitativa,nominal
3,age,idade em anos,quantitativa,discreta
4,sibsp,# de irmãos/cônjuges a bordo do Titanic,quantitativa,discreta
5,parch,"# de pais/crianças a bordo do Titanic, 0 para ...",quantitativa,discreta
6,fare,Tarifa de passageiro,quantitativa,continua
7,embarked,Ponto de embarcacao,qualitativa,nominal
8,class,Se refere a classe social ou categoria que os ...,qualitativa,ordinal
9,who,informacao do genero (desnecessaria pois ja te...,qualitativa,nominal


---

## 2. Preparação de dados

### 2.1 Imputando Valores

In [3]:
df['age'].fillna(df['age'].median(), inplace=True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)
df['embark_town'].fillna(df['embark_town'].mode()[0], inplace=True)

---

### 2.2 Removendo Variáveis Desnecessárias 

In [4]:
df.drop(columns=['alive', 'embarked', 'class', 'deck', 'who'], inplace=True)

##### A remoção da variável `who`, `deck`, `alive`, `embarked` e `class` ocorreram pois alive, embarked, class e who causam redundância e deck tem excesso de valores faltantes.

### 2.3 Tratamento de dados discrepantes

In [5]:
Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['fare'] = df['fare'].clip(lower=lower_bound, upper=upper_bound)

---

### 2.4 Normalização

In [6]:
# Definir as variáveis numéricas
numerical_features = ['age', 'sibsp', 'parch', 'fare']

# Criar um pipeline para pré-processamento das variáveis numéricas
numerical_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())  # Normalizar os dados para ter média 0 e desvio padrão 1
])

# Aplicar a normalização às variáveis numéricas
X_numerical_processed = numerical_pipeline.fit_transform(df[numerical_features])

##### A normalização dos dados foi realizada para garantir que todas as variáveis numéricas tivessem a mesma escala e, assim, fossem comparáveis entre si. Utilizou-se o StandardScaler do `scikit-learn`, que transformou as variáveis numéricas `age`, `sibsp`, `parch` e `fare` para que tivessem média 0 e desvio padrão 1. Essa etapa é crucial para que as variáveis contribuem igualmente durante o treinamento de modelos de aprendizado de máquina.

---

### 2.5 Codificação

In [10]:
# Definir as variáveis categóricas e ordinais
categorical_features = ['sex', 'embark_town', 'adult_male', 'alone']
ordinal_features = ['pclass']

# Criar um pipeline para pré-processamento das variáveis categóricas
categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificar as variáveis categóricas com one-hot encoding
])

# Criar um preprocessor para aplicar as transformações às variáveis
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_pipeline, categorical_features),
        ('ord', 'passthrough', ordinal_features)  # Manter as variáveis ordinais como estão
    ]
)

# Aplicar a codificação às variáveis categóricas e ordinais
X_categorical_processed = preprocessor.fit_transform(df)

##### Para preparar as variáveis categóricas e ordinais para o modelo, foi aplicada a codificação. As variáveis categóricas, como `sex`, `adult_male` e `alone`, foram transformadas utilizando o `OneHotEncoder` do `scikit-learn`, que converteu cada categoria em múltiplas colunas binárias, permitindo que o modelo interprete cada categoria de forma adequada. Já as variáveis ordinais, como `pclass`, foram mantidas em sua forma original com o uso do `'passthrough'`, preservando a ordem natural dos valores. Essas transformações asseguram que o modelo possa processar todas as variáveis corretamente.


---